## Euro USD data set

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [2]:
data_usd_euro = pd.read_csv("csv files/EUR_USD.csv")
data_usd_euro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2870 entries, 0 to 2869
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   2870 non-null   object
 1   Şimdi   2870 non-null   object
 2   Açılış  2870 non-null   object
 3   Yüksek  2870 non-null   object
 4   Düşük   2870 non-null   object
 5   Hac.    1041 non-null   object
 6   Fark %  2870 non-null   object
dtypes: object(7)
memory usage: 157.1+ KB


In [3]:
df_euro = data_usd_euro.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [4]:
# We need to change from "," to "." because it will be a float. 
df_euro['Şimdi'] = df_euro['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.'))/20)
df_euro

,Tarih,Şimdi
0,30.12.2022,1.0702
1,29.12.2022,1.0661
2,28.12.2022,1.0608
3,27.12.2022,1.0638
4,26.12.2022,1.0635
...,...,...
2865,06.01.2012,1.2721
2866,05.01.2012,1.2780
2867,04.01.2012,1.2942
2868,03.01.2012,1.3051


## LSTM

In [5]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df_euro['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.0702 1.0661 1.0608 1.0638 1.0635] 1.0614
[1.0661 1.0608 1.0638 1.0635 1.0614] 1.0593
[1.0608 1.0638 1.0635 1.0614 1.0593] 1.0603
[1.0638 1.0635 1.0614 1.0593 1.0603] 1.0621
[1.0635 1.0614 1.0593 1.0603 1.0621] 1.0605
[1.0614 1.0593 1.0603 1.0621 1.0605] 1.0582
[1.0593 1.0603 1.0621 1.0605 1.0582] 1.0626
[1.0603 1.0621 1.0605 1.0582 1.0626] 1.0682
[1.0621 1.0605 1.0582 1.0626 1.0682] 1.063
[1.0605 1.0582 1.0626 1.0682 1.063 ] 1.0535
[1.0582 1.0626 1.0682 1.063  1.0535] 1.053
[1.0626 1.0682 1.063  1.0535 1.053 ] 1.0556
[1.0682 1.063  1.0535 1.053  1.0556] 1.0505
[1.063  1.0535 1.053  1.0556 1.0505] 1.0469
[1.0535 1.053  1.0556 1.0505 1.0469] 1.0491
[1.053  1.0556 1.0505 1.0469 1.0491] 1.0538
[1.0556 1.0505 1.0469 1.0491 1.0538] 1.0522
[1.0505 1.0469 1.0491 1.0538 1.0522] 1.0405
[1.0469 1.0491 1.0538 1.0522 1.0405] 1.0327
[1.0491 1.0538 1.0522 1.0405 1.0327] 1.0337
[1.0538 1.0522 1.0405 1.0327 1.0337] 1.0395
[1.0522 1.0405 1.0327 1.0337 1.0395] 1.0408
[1.0405 1.0327 1.0337 1.0395 1.040

In [6]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df_euro['Şimdi'][0],df_euro['Şimdi'][1],df_euro['Şimdi'][2],df_euro['Şimdi'][3],df_euro['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
90/90 [==============================] - 1s 3ms/step - loss: 0.2540
Epoch 2/200
90/90 [==============================] - 0s 3ms/step - loss: 1.2378e-04
Epoch 3/200
90/90 [==============================] - 0s 3ms/step - loss: 1.0207e-04
Epoch 4/200
90/90 [==============================] - 0s 3ms/step - loss: 9.9233e-05
Epoch 5/200
90/90 [==============================] - 0s 3ms/step - loss: 9.7859e-05
Epoch 6/200
90/90 [==============================] - 0s 3ms/step - loss: 9.4000e-05
Epoch 7/200
90/90 [==============================] - 0s 3ms/step - loss: 9.5755e-05
Epoch 8/200
90/90 [==============================] - 0s 3ms/step - loss: 9.2766e-05
Epoch 9/200
90/90 [==============================] - 0s 3ms/step - loss: 9.2261e-05
Epoch 10/200
90/90 [==============================] - 0s 3ms/step - loss: 9.2274e-05
Epoch 11/200
90/90 [==============================] - 0s 3ms/step - loss: 9.3901e-05
Epoch 12/200
90/90 [==============================] - 0s 3ms/step - loss: 9.33

## Daily

In [7]:
x = datetime.datetime.now()
print("Prediction:",round(yhat[0][0]*20,3), "Euro/USD in", x.strftime('%x'))

Prediction: 1.065 Euro/USD in 01/01/23


## Weekly

In [11]:
x_input = array([df_euro['Şimdi'][0], df_euro['Şimdi'][1], df_euro['Şimdi'][2], df_euro['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df_euro['Şimdi'][0], df_euro['Şimdi'][1], df_euro['Şimdi'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df_euro['Şimdi'][0], df_euro['Şimdi'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df_euro['Şimdi'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)



yhat = float(yhat)*20
yhat_2 = float(yhat_2)*20
yhat_3 = float(yhat_3)*20
yhat_4 = float(yhat_4)*20
yhat_5 = float(yhat_5)*20


In [12]:
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  $ Euro/USD ")


Monday: 1.065 Tuesday : 1.066 Wednesday: 1.066 Thursday: 1.066 Friday: 1.066  $ Euro/USD 


## Monthly

In [13]:
import array
exchange = array.array('f', [])

for i in range(len(df_euro)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df_euro['Şimdi'][i])


In [14]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.0702 1.0614 1.0582 1.053  1.0538] 1.0394999980926514
[1.0614 1.0582 1.053  1.0538 1.0395] 1.0324000120162964
[1.0582 1.053  1.0538 1.0395 1.0324] 1.0351999998092651
[1.053  1.0538 1.0395 1.0324 1.0352] 0.9959999918937683
[1.0538 1.0395 1.0324 1.0352 0.996 ] 0.9962999820709229
[1.0395 1.0324 1.0352 0.996  0.9963] 0.9860000014305115
[1.0324 1.0352 0.996  0.9963 0.986 ] 0.9718999862670898
[1.0352 0.996  0.9963 0.986  0.9719] 0.9740999937057495
[0.996  0.9963 0.986  0.9719 0.9741] 0.9799000024795532
[0.9963 0.986  0.9719 0.9741 0.9799] 0.968999981880188
[0.986  0.9719 0.9741 0.9799 0.969 ] 1.0015000104904175
[0.9719 0.9741 0.9799 0.969  1.0015] 1.0039000511169434
[0.9741 0.9799 0.969  1.0015 1.0039] 0.9951000213623047
[0.9799 0.969  1.0015 1.0039 0.9951] 0.9961000084877014
[0.969  1.0015 1.0039 0.9951 0.9961] 1.0033999681472778
[1.0015 1.0039 0.9951 0.9961 1.0034] 1.0257999897003174
[1.0039 0.9951 0.9961 1.0034 1.0258] 1.0181000232696533
[0.9951 0.9961 1.0034 1.0258 1.0181] 1.0218000411

In [15]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/200
18/18 [==============================] - 1s 5ms/step - loss: 0.4882
Epoch 2/200
18/18 [==============================] - 0s 5ms/step - loss: 0.0267
Epoch 3/200
18/18 [==============================] - 0s 5ms/step - loss: 0.0047
Epoch 4/200
18/18 [==============================] - 0s 6ms/step - loss: 0.0010
Epoch 5/200
18/18 [==============================] - 0s 5ms/step - loss: 5.0206e-04
Epoch 6/200
18/18 [==============================] - 0s 5ms/step - loss: 4.6464e-04
Epoch 7/200
18/18 [==============================] - 0s 5ms/step - loss: 4.0498e-04
Epoch 8/200
18/18 [==============================] - 0s 5ms/step - loss: 4.1984e-04
Epoch 9/200
18/18 [==============================] - 0s 5ms/step - loss: 4.0849e-04
Epoch 10/200
18/18 [==============================] - 0s 5ms/step - loss: 3.9420e-04
Epoch 11/200
18/18 [==============================] - 0s 5ms/step - loss: 4.0334e-04
Epoch 12/200
18/18 [==============================] - 0s 5ms/step - loss: 4.0141e-04
Epoch

In [16]:
# onluk olarak yukarı yuvarlandı.
print("Next Month -> Prediction:", round(float(yhat[0][0]*20),3),  "$ Euro/USD in", x.strftime('%B'))


Next Month -> Prediction: 1.074 $ Euro/USD in January


## Yearly

In [17]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df_euro)):
    if i % 10 == 0:
        exchange_2.append(df_euro['Şimdi'][i])


In [18]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.0702 1.0324 0.9741 0.9961 1.0087] 1.0717999935150146
[1.0324 0.9741 0.9961 1.0087 1.0718] 1.0793999433517456
[0.9741 0.9961 1.0087 1.0718 1.0794] 1.09089994430542
[0.9961 1.0087 1.0718 1.0794 1.0909] 1.114300012588501
[1.0087 1.0718 1.0794 1.0909 1.1143] 1.123900055885315
[1.0718 1.0794 1.0909 1.1143 1.1239] 1.156599998474121
[1.0794 1.0909 1.1143 1.1239 1.1566] 1.1713999509811401
[1.0909 1.1143 1.1239 1.1566 1.1714] 1.1791000366210938
[1.1143 1.1239 1.1566 1.1714 1.1791] 1.186400055885315
[1.1239 1.1566 1.1714 1.1791 1.1864] 1.217900037765503
[1.1566 1.1714 1.1791 1.1864 1.2179] 1.1895999908447266
[1.1714 1.1791 1.1864 1.2179 1.1896] 1.2073999643325806
[1.1791 1.1864 1.2179 1.1896 1.2074] 1.207800030708313
[1.1864 1.2179 1.1896 1.2074 1.2078] 1.2120000123977661
[1.2179 1.1896 1.2074 1.2078 1.212 ] 1.1858999729156494
[1.1896 1.2074 1.2078 1.212  1.1859] 1.184499979019165
[1.2074 1.2078 1.212  1.1859 1.1845] 1.17739999294281
[1.2078 1.212  1.1859 1.1845 1.1774] 1.1174999475479126
[1.

In [19]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)

# demonstrate prediction
x_input = array([exchange_2[0], exchange_2[1],
                exchange_2[2], exchange_2[3], exchange_2[4]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/200
3/3 [==============================] - 1s 5ms/step - loss: 1.3531
Epoch 2/200
3/3 [==============================] - 0s 6ms/step - loss: 1.0264
Epoch 3/200
3/3 [==============================] - 0s 6ms/step - loss: 0.7442
Epoch 4/200
3/3 [==============================] - 0s 6ms/step - loss: 0.4612
Epoch 5/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1744
Epoch 6/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0230
Epoch 7/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1391
Epoch 8/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0636
Epoch 9/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0068
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0286
Epoch 11/200
3/3 [==============================] - 0s 5ms/step - loss: 0.0422
Epoch 12/200
3/3 [==============================] - 0s 6ms/step - loss: 0.0285
Epoch 13/200
3/3 [==============================] - 0s 5ms/st

In [20]:
print("Next Year -> Prediction:" ,round(float(yhat[0][0]*20),3), "$ Euro/USD in", x.strftime('%Y'))


Next Year -> Prediction: 1.055 $ Euro/USD in 2023
